# ML-5 : Time Series Forecasting avec ML.NET

**Navigation** : [<< ML-4-Evaluation](ML-4-Evaluation.ipynb) | [ML-6-ONNX](ML-6-ONNX.ipynb) | [Index](README.md)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre les principes du **forecasting** de séries temporelles
2. Utiliser **ForecastBySsa** (Singular Spectrum Analysis) pour la prévision
3. Détecter la **saisonnalité** dans les données temporelles
4. Interpréter les **intervalles de confiance** des prévisions
5. Comparer différentes configurations de modèles de forecasting

### Prérequis
- .NET SDK 9.0
- Notebooks ML-1 à ML-4 complétées
- Connaissances de base en séries temporelles

### Durée estimée : 45-60 minutes

---

# Introduction au Time Series Forecasting

Le **forecasting** (prévision) de séries temporelles consiste à prédire les valeurs futures d'une variable basée sur ses valeurs passées.

## Concepts clés

| Concept | Description |
|---------|-------------|
| **Trend** | Tendance à long terme (croissance, décroissance) |
| **Saisonnalité** | Pattern périodique (hebdomadaire, mensuel, annuel) |
| **Bruit** | Variations aléatoires non prévisibles |
| **Horizon** | Nombre de pas de temps à prédire |
| **Intervalle de confiance** | Plage de valeurs probables (95%) |

## Singular Spectrum Analysis (SSA)

**ForecastBySsa** utilise l'algorithme SSA qui décompose la série temporelle en composantes :

```
Série Temporelle = Trend + Saisonnalité + Bruit
```

**Avantages de SSA** :
- Fonctionne avec des données non stationnaires
- Capture automatiquement la saisonnalité
- Fournit des intervalles de confiance
- Peu de paramètres à régler

In [ ]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.TimeSeries"
#r "nuget: XPlot.Plotly.Interactive"

using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;
using System.Collections.Generic;

Console.WriteLine("Packages chargés avec succès !");

Les packages nécessaires sont :
- **Microsoft.ML** : framework principal de machine learning
- **Microsoft.ML.TimeSeries** : algorithmes de séries temporelles (ForecastBySsa)
- **XPlot.Plotly.Interactive** : visualisations interactives

## Partie 1 : Chargement et exploration des données

Nous utilisons un dataset de **ventes quotidiennes** sur 2 ans (2023-2024) avec une tendance et une saisonnalité hebdomadaire.

### Structure du dataset (`daily-sales.csv`)

| Colonne | Type | Description |
|---------|------|-------------|
| Date | DateTime | Date de la vente |
| Year | int | Année (2023 ou 2024) |
| Month | int | Mois (1-12) |
| Day | int | Jour du mois (1-31) |
| DayOfWeek | int | Jour de la semaine (0=Lundi, 6=Dimanche) |
| Sales | int | Nombre de ventes |

In [ ]:
// Définition du schéma de données
public class DailySalesData
{
    public DateTime Date { get; set; }
    public float Year { get; set; }
    public float Month { get; set; }
    public float Day { get; set; }
    public float DayOfWeek { get; set; }
    public float Sales { get; set; }
}

public class SalesForecast
{
    public float[] ForecastedSales { get; set; }
    public float[] LowerBoundSales { get; set; }
    public float[] UpperBoundSales { get; set; }
}

// Initialiser MLContext
var mlContext = new MLContext(seed: 42);

// Charger les données
var dataPath = Path.Combine(Environment.CurrentDirectory, "daily-sales.csv");
Console.WriteLine($"Chargement des données depuis {dataPath}");

var fullData = mlContext.Data.LoadFromTextFile<DailySalesData>(
    path: dataPath,
    hasHeader: true,
    separatorChar: ',');

// Afficher les statistiques
var dataStats = mlContext.Data.CreateEnumerable<DailySalesData>(fullData, reuseRowObject: false);
var totalDays = dataStats.Count();
var totalSales = dataStats.Sum(x => x.Sales);
var avgSales = dataStats.Average(x => x.Sales);
var minDate = dataStats.Min(x => x.Date);
var maxDate = dataStats.Max(x => x.Date);

Console.WriteLine($"\n=== Statistiques du dataset ===");
Console.WriteLine($"Période : {minDate:yyyy-MM-dd} à {maxDate:yyyy-MM-dd}");
Console.WriteLine($"Nombre total de jours : {totalDays}");
Console.WriteLine($"Ventes totales : {totalSales:N0}");
Console.WriteLine($"Ventes moyennes par jour : {avgSales:F1}");

// Afficher les premières lignes
Console.WriteLine("\n=== 10 premières lignes ===");
var firstRows = mlContext.Data.CreateEnumerable<DailySalesData>(fullData, reuseRowObject: false)
    .Take(10)
    .Select(x => $"{x.Date:yyyy-MM-dd} | Year: {x.Year} | DayOfWeek: {x.DayOfWeek} | Sales: {x.Sales}");

foreach (var row in firstRows)
{
    Console.WriteLine(row);
}

### Interprétation des statistiques

| Métrique | Ce que cela signifie |
|----------|----------------------|
| Période 2023-2024 | 2 années de données pour capturer la saisonnalité |
| ~730 jours | Données quotidiennes pour une analyse fine |
| Ventes moyennes | Niveau de base autour duquel les ventes oscillent |

**Analyse visuelle** : Nous allons visualiser les données pour identifier la tendance et la saisonnalité.

## Partie 2 : Visualisation de la série temporelle

Visualisons les ventes sur la période complète pour identifier :
- La **tendance** à long terme
- La **saisonnalité** hebdomadaire
- Les **anomalies** éventuelles

In [ ]:
// Extraire les données pour la visualisation
var allData = mlContext.Data.CreateEnumerable<DailySalesData>(fullData, reuseRowObject: false)
    .OrderBy(x => x.Date)
    .ToArray();

var dates = allData.Select(x => x.Date.ToString("yyyy-MM-dd")).ToArray();
var sales = allData.Select(x => (double)x.Sales).ToArray();

// Créer le graphique
var trace = new Scatter
{
    x = dates,
    y = sales,
    mode = "lines",
    name = "Ventes quotidiennes",
    line = new Line { width = 1 }
};

var layout = new Layout.Layout
{
    title = "Ventes quotidiennes (2023-2024)",
    xaxis = new Xaxis { title = "Date" },
    yaxis = new Yaxis { title = "Nombre de ventes" },
    height = 400,
    width = 1200
};

var chart = Chart.Plot(trace, layout);
display(chart);

### Interprétation du graphique

**Observations attendues** :

| Pattern | Description |
|---------|-------------|
| Tendance croissante | Les ventes augmentent progressivement de 2023 à 2024 |
| Oscillations | Pattern hebdomadaire visible (pics et creux) |
| Variabilité | Fluctuations aléatoires autour de la tendance |

> **Question** : Identifiez visuellement la période des pics de ventes. Est-ce hebdomadaire ?

## Partie 3 : Prévision avec ForecastBySsa

Nous allons utiliser **ForecastBySsa** pour prédire les ventes futures.

### Paramètres clés

| Paramètre | Valeur | Description |
|-----------|--------|-------------|
| `windowSize` | 7 | Taille de la fenêtre pour l'analyse SSA (7 = saisonnalité hebdomadaire) |
| `seriesLength` | 30 | Longueur de la série utilisée pour chaque prévision |
| `trainSize` | 365 | Nombre de points d'entraînement (1 année) |
| `horizon` | 7 | Nombre de jours à prédire (1 semaine) |
| `confidenceLevel` | 0.95 | Niveau de confiance pour les intervalles (95%) |

In [ ]:
using Microsoft.ML.Transforms.TimeSeries;

// Séparer les données : entraînement sur 2023, test sur 2024
var trainData = mlContext.Data.FilterRowsByColumn(fullData, "Year", upperBound: 2024);
var testData = mlContext.Data.FilterRowsByColumn(fullData, "Year", lowerBound: 2024);

Console.WriteLine("=== Division des données ===");
Console.WriteLine($"Entraînement (2023) : {mlContext.Data.CreateEnumerable<DailySalesData>(trainData, reuseRowObject: false).Count()} jours");
Console.WriteLine($"Test (2024) : {mlContext.Data.CreateEnumerable<DailySalesData>(testData, reuseRowObject: false).Count()} jours");

// Définir le pipeline de forecasting
var forecastingPipeline = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastedSales",
    inputColumnName: "Sales",
    windowSize: 7,              // Fenêtre de 7 jours (saisonnalité hebdomadaire)
    seriesLength: 30,           // Utiliser 30 jours pour la prévision
    trainSize: 365,             // Entraîner sur 1 année
    horizon: 7,                 // Prédire 7 jours à l'avance
    confidenceLevel: 0.95f,     // Intervalle de confiance à 95%
    confidenceLowerBoundColumn: "LowerBoundSales",
    confidenceUpperBoundColumn: "UpperBoundSales");

Console.WriteLine("\n=== Entraînement du modèle ForecastBySsa ===");
var forecaster = forecastingPipeline.Fit(trainData);
Console.WriteLine("✓ Modèle entraîné avec succès");

### Interprétation des paramètres

| Paramètre | Impact |
|-----------|--------|
| `windowSize: 7` | Capture la saisonnalité hebdomadaire |
| `seriesLength: 30` | Utilise le dernier mois pour prédire la semaine suivante |
| `trainSize: 365` | Apprentissage sur 1 année complète |
| `horizon: 7` | Prévisions à 7 jours (court terme) |

> **Note** : Ces paramètres peuvent être ajustés selon vos données et vos besoins de prévision.

## Partie 4 : Évaluation des prévisions

Évaluons la qualité des prévisions en comparant les valeurs prédites aux valeurs réelles sur l'année 2024.

In [ ]:
// Faire des prédictions sur les données de test
var predictions = forecaster.Transform(testData);

// Extraire les valeurs réelles et prédites
var actual = mlContext.Data.CreateEnumerable<DailySalesData>(testData, reuseRowObject: false)
    .Select(x => (double)x.Sales)
    .ToArray();

var forecast = mlContext.Data.CreateEnumerable<SalesForecast>(predictions, reuseRowObject: false)
    .SelectMany(x => x.ForecastedSales)
    .ToArray();

// Calculer les métriques d'erreur
var errors = actual.Zip(forecast, (a, f) => a - f);
var mae = errors.Average(e => Math.Abs(e));
var rmse = Math.Sqrt(errors.Average(e => e * e));

Console.WriteLine("=== Métriques d'évaluation ===");
Console.WriteLine($"Mean Absolute Error (MAE): {mae:F2} ventes");
Console.WriteLine($"Root Mean Squared Error (RMSE): {rmse:F2} ventes");
Console.WriteLine($"\nInterprétation :");
Console.WriteLine($"  - MAE : En moyenne, les prévisions s'écartent de {mae:F1} ventes");
Console.WriteLine($"  - RMSE : Les erreurs importantes sont pénalisées davantage");

### Interprétation des métriques

| Métrique | Bonne performance | Mauvaise performance |
|----------|-------------------|----------------------|
| MAE faible | < 10% des ventes moyennes | > 30% des ventes moyennes |
| RMSE proche de MAE | Erreurs uniformes | Grosses erreurs ponctuelles |

**Exemple** : Si les ventes moyennes sont de 150, un MAE de 15 (10%) est considéré comme bon.

## Partie 5 : Visualisation des prévisions

Comparons les prévisions aux valeurs réelles pour les 30 premiers jours de 2024.

In [ ]:
// Extraire les 30 premiers jours de 2024
var comparisonData = mlContext.Data.CreateEnumerable<DailySalesData>(testData, reuseRowObject: false)
    .OrderBy(x => x.Date)
    .Take(30)
    .ToArray();

var comparisonDates = comparisonData.Select(x => x.Date.ToString("MM-dd")).ToArray();
var comparisonActual = comparisonData.Select(x => (double)x.Sales).ToArray();

// Récupérer les prévisions correspondantes
var comparisonForecast = mlContext.Data.CreateEnumerable<SalesForecast>(predictions, reuseRowObject: false)
    .Take(30)
    .SelectMany(x => x.ForecastedSales)
    .ToArray();

// Créer le graphique de comparaison
var actualTrace = new Scatter
{
    x = comparisonDates,
    y = comparisonActual,
    mode = "lines+markers",
    name = "Ventes réelles",
    line = new Line { color = "blue" }
};

var forecastTrace = new Scatter
{
    x = comparisonDates,
    y = comparisonForecast,
    mode = "lines+markers",
    name = "Prévisions",
    line = new Line { color = "orange", dash = "dash" }
};

var layout = new Layout.Layout
{
    title = "Comparaison : Réel vs Prévisions (30 premiers jours de 2024)",
    xaxis = new Xaxis { title = "Date (2024)" },
    yaxis = new Yaxis { title = "Nombre de ventes" },
    height = 400,
    width = 1000
};

var chart = Chart.Plot(new[] { actualTrace, forecastTrace }, layout);
display(chart);

### Interprétation du graphique

**Analyse visuelle** :

| Observation | Signification |
|-------------|---------------|
| Lignes parallèles | Bonne capture de la tendance |
| Décalage constant | Le modèle suit bien les variations |
| Croisements fréquents | Le modèle n'anticipe pas bien les changements |

> **Question** : Les prévisions sont-elles systématiquement au-dessus ou en-dessous des valeurs réelles ?

## Partie 6 : Intervalles de confiance

ForecastBySsa fournit des **intervalles de confiance** qui quantifient l'incertitude des prévisions.

In [ ]:
// Extraire les intervalles de confiance pour les 7 premiers jours
var forecastWithBounds = mlContext.Data.CreateEnumerable<SalesForecast>(predictions, reuseRowObject: false)
    .First();

Console.WriteLine("=== Prévisions à 7 jours avec intervalles de confiance (95%) ===");
Console.WriteLine("\nDate       | Réel | Prévision | Borne inf | Borne sup | Largeur");
Console.WriteLine(new string('-', 80));

var testDates = mlContext.Data.CreateEnumerable<DailySalesData>(testData, reuseRowObject: false)
    .OrderBy(x => x.Date)
    .Take(7)
    .ToArray();

for (int i = 0; i < 7; i++)
{
    var date = testDates[i].Date.ToString("yyyy-MM-dd");
    var actual = testDates[i].Sales;
    var forecast = forecastWithBounds.ForecastedSales[i];
    var lower = Math.Max(0, forecastWithBounds.LowerBoundSales[i]);
    var upper = forecastWithBounds.UpperBoundSales[i];
    var width = upper - lower;
    var inBounds = actual >= lower && actual <= upper ? "✓" : "✗";
    
    Console.WriteLine($"{date} | {actual,4} | {forecast,9:F1} | {lower,9:F1} | {upper,9:F1} | {width,6:F1} {inBounds}");
}

### Interprétation des intervalles de confiance

| Concept | Description |
|---------|-------------|
| **Borne inférieure** | Pire cas réaliste (2.5ème percentile) |
| **Borne supérieure** | Meilleur cas réaliste (97.5ème percentile) |
| **Largeur** | Incertitude de la prévision (plus large = plus incertain) |
| **✓** | La valeur réelle est dans l'intervalle (bon signe) |

> **Règle empirique** : Si > 95% des valeurs réelles sont dans l'intervalle, le modèle est bien calibré.

## Partie 7 : Comparaison de configurations

Comparons différentes configurations de ForecastBySsa pour trouver la meilleure.

In [ ]:
// Tester différentes configurations
var configs = new[]
{
    new { windowSize = 7, seriesLength = 30, Name = "Config 1: w=7, s=30 (base)" },
    new { windowSize = 14, seriesLength = 30, Name = "Config 2: w=14, s=30 (saisonnalité 2 sem)" },
    new { windowSize = 7, seriesLength = 60, Name = "Config 3: w=7, s=60 (plus d'historique)" }
};

Console.WriteLine("=== Comparaison des configurations ===");
Console.WriteLine("\nConfiguration | MAE | RMSE");
Console.WriteLine(new string('-', 60));

foreach (var config in configs)
{
    var pipeline = mlContext.Forecasting.ForecastBySsa(
        outputColumnName: "ForecastedSales",
        inputColumnName: "Sales",
        windowSize: config.windowSize,
        seriesLength: config.seriesLength,
        trainSize: 365,
        horizon: 7,
        confidenceLevel: 0.95f,
        confidenceLowerBoundColumn: "LowerBoundSales",
        confidenceUpperBoundColumn: "UpperBoundSales");
    
    var model = pipeline.Fit(trainData);
    var preds = model.Transform(testData);
    
    var actualVals = mlContext.Data.CreateEnumerable<DailySalesData>(testData, reuseRowObject: false)
        .Select(x => (double)x.Sales).ToArray();
    var forecastVals = mlContext.Data.CreateEnumerable<SalesForecast>(preds, reuseRowObject: false)
        .SelectMany(x => x.ForecastedSales).ToArray();
    
    var error = actualVals.Zip(forecastVals, (a, f) => a - f);
    var mae = error.Average(e => Math.Abs(e));
    var rmse = Math.Sqrt(error.Average(e => e * e));
    
    Console.WriteLine($"{config.Name,-45} | {mae,5:F1} | {rmse,5:F1}");
}

### Interprétation des résultats

| Configuration | Quand l'utiliser ? |
|---------------|-------------------|
| windowSize=7 | Saisonnalité hebdomadaire (ventes, trafic) |
| windowSize=14 | Cycle bi-hebdomadaire (paye, etc.) |
| seriesLength=30 | Prévisions court terme |
| seriesLength=60 | Plus de contexte, prévisions plus stables |

> **Recommandation** : Choisissez la configuration avec le MAE le plus faible.

## Conclusion et points clés

Ce notebook a couvert les principes du **forecasting** avec ML.NET :

| Concept | Point clé |
|---------|-----------|
| **ForecastBySsa** | Algorithme SSA pour décomposition trend/saisonnalité/bruit |
| **Paramètres** | windowSize capture la saisonnalité, horizon = horizon de prévision |
| **Intervalles de confiance** | Quantifient l'incertitude des prévisions |
| **Évaluation** | MAE et RMSE pour mesurer la précision |

**Prochaines étapes** :
- Essayer différents paramètres selon vos données
- Tester sur d'autres types de séries temporelles
- Explorer d'autres algorithmes (AutoML pour TimeSeries)

**Navigation** : [<< ML-4-Evaluation](ML-4-Evaluation.ipynb) | [ML-6-ONNX](ML-6-ONNX.ipynb) | [Index](README.md)